# DATASET

*filename:* idiom_all_repository_all.json

## schema and sample row

|variations |idiom|sources |entry | id | confidence |
|---------------|-----------------|-----------------|----------------|--------|--------|
| *varchar[] | *varchar* | *varchar[]* | *json (varchar in df)* | *int64* |*int64* |
| [] | 'er indoors | [Wiktionary] | [{"usages":[[]],"definition":"one's wife","pos":"noun"}]    | 1 | 1

### variations 
list of phrase variations of idiom
- can be empty list []

### idiom
the phrase
- can NOT be NULL
  
### sources
list of sources for idiom
- can NOT be NULL

### id
unique identifier for entry/row
- can NOT be NULL
  
### confidence
dataset confidence in idiom information
- can NOT be NULL
- possible values (in curr. dataset): [1, 2, 3]
- seems to correspond to length of sources list (only exception id# 6674)

## size
- 9721 unique rows
- 4.9 MB

In [17]:
import duckdb as db

FILE_PATH = '../Data/idiom_repository_all.json'

idiom_df = db.query(f"""
    SELECT *
    FROM read_json_auto('{FILE_PATH}')
""").df()

In [19]:
db.query(f"""
    SELECT *
    FROM read_json_auto('{FILE_PATH}')
""")

┌────────────┬─────────────────────────────────┬───────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬───────┬────────────┐
│ variations │              idiom              │        sources        │                                                                                                                                                                                                                                                                       entry                                  

In [18]:
idiom_df

,variations,idiom,sources,entry,id,confidence
0,[],'er indoors,[Wiktionary],"[{""usages"":[[]],""definition"":""one's wife"",""pos...",1,1
1,[],10 Downing Street,[Wiktionary],"[{""usages"":[[""... explicitly clear, in the way...",2,1
2,[],11 Downing Street,[Wiktionary],"[{""usages"":[[]],""definition"":""w treasury of th...",3,1
3,[],110 proof,[Wiktionary],"[{""usages"":[[""in those days a hollywood hero h...",4,1
4,[],12-ounce curls,[Wiktionary],"[{""usages"":[[""don't burn out your biceps with ...",5,1
...,...,...,...,...,...,...
9716,[],words fail me,[Learn English Today],"""""",9874,1
9717,[],bandit territory,[Learn English Today],"""""",9875,1
9718,[],beat black and blue,[Learn English Today],"""""",9876,1
9719,[],free-for-all,[Learn English Today],"""""",9877,1


In [ ]:
# number of unique datapoints

db.query(f"""
    SELECT DISTINCT count(*) as num_rows
    FROM idiom_df
""")

┌──────────┐
│ num_rows │
│  int64   │
├──────────┤
│     9721 │
└──────────┘

In [27]:
db.query(f"""
    SELECT *
    FROM idiom_df 
    WHERE confidence IS NULL
""")

┌────────────┬─────────┬───────────┬─────────┬───────┬────────────┐
│ variations │  idiom  │  sources  │  entry  │  id   │ confidence │
│ varchar[]  │ varchar │ varchar[] │ varchar │ int64 │   int64    │
├────────────┴─────────┴───────────┴─────────┴───────┴────────────┤
│                             0 rows                              │
└─────────────────────────────────────────────────────────────────┘

In [24]:
db.query(f"""
    SELECT DISTINCT confidence, count(*)
    FROM idiom_df 
    GROUP BY confidence
""")

┌────────────┬──────────────┐
│ confidence │ count_star() │
│   int64    │    int64     │
├────────────┼──────────────┤
│          2 │          873 │
│          1 │         8758 │
│          3 │           90 │
└────────────┴──────────────┘

In [41]:
db.query(f"""
    WITH num_sources AS (
        SELECT id, len(sources) as len
        FROM idiom_df 
    )   
    SELECT df.id, ns.len, df.confidence, (ns.len == df.confidence) as matches
    FROM num_sources ns
    LEFT JOIN idiom_df df ON df.id == ns.id
    WHERE matches IS FALSE
""")

┌───────┬───────┬────────────┬─────────┐
│  id   │  len  │ confidence │ matches │
│ int64 │ int64 │   int64    │ boolean │
├───────┼───────┼────────────┼─────────┤
│  6674 │     2 │          1 │ false   │
└───────┴───────┴────────────┴─────────┘

In [42]:
db.query(f"""
    SELECT *
    FROM idiom_df 
    WHERE id == 6674
""")

┌─────────────────────────────┬────────────────────────────┬───────────────────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────